In [ ]:
# Import the required packages and functions
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time

from ml_control.greedy_algorithm import greedy
from ml_control.problem_definitions.heat_equation import create_heat_equation_problem_complex
from ml_control.systems import solve_optimal_control_problem

In [ ]:
# Define the system components
system_dimension = 50
T, nt, N, h, parametrized_A, parametrized_B, parametrized_x0, parametrized_xT, R_chol, M, parameter_space = create_heat_equation_problem_complex(system_dimension)
spatial_norm = lambda x: np.linalg.norm(h * x)
temporal_norm = lambda u: np.linalg.norm(u * (T / nt))

In [ ]:
# Run the greedy algorithm
k_train = 8
training_parameters = np.array(np.meshgrid(np.linspace(*parameter_space[0], k_train),
                                           np.linspace(*parameter_space[1], k_train))).T.reshape(-1, 2)
tol = 1e-6
max_basis_size = k_train ** 2

_, reduced_basis, _, estimated_errors, true_errors, efficiencies, optimal_adjoints, training_data = greedy(training_parameters, N, T, nt, parametrized_A, parametrized_B, parametrized_x0, parametrized_xT, R_chol, M,
                                                                                                  tol=tol, max_basis_size=max_basis_size, return_errors_and_efficiencies=True, spatial_norm=spatial_norm)

In [ ]:
_, singular_values, _ = np.linalg.svd(optimal_adjoints)

In [ ]:
with open("greedy_results.pkl", "wb") as f:
    pickle.dump((reduced_basis, estimated_errors, true_errors, efficiencies, training_data, singular_values, tol), f)

In [ ]:
k_test_analysis = 25
test_parameters_analysis = np.stack([np.random.uniform(np.array(parameter_space)[0,0], np.array(parameter_space)[0,1], k_test_analysis),
                                     np.random.uniform(np.array(parameter_space)[1,0], np.array(parameter_space)[1,1], k_test_analysis)]).T
test_parameters_analysis = np.sort(test_parameters_analysis, axis=0)
test_solutions_analysis = []
for mu in test_parameters_analysis:
    A = parametrized_A(mu)
    B = parametrized_B(mu)
    x0 = parametrized_x0(mu)
    xT = parametrized_xT(mu)
    phiT_init = np.zeros(N)
    phi_opt = solve_optimal_control_problem(x0, xT, T, nt, A, B, R_chol, M, phiT_init)
    test_solutions_analysis.append(phi_opt)

In [ ]:
with open("test_data.pkl", "wb") as f:
    pickle.dump((test_parameters_analysis, test_solutions_analysis), f)